In [1]:
import torch
import torchvision.models as models
!pip install onnx onnxruntime
import onnx
import onnxruntime

In [2]:
#!pip uninstall onnx2torch
#!pip install onnx2torch
!pip install -e git+https://github.com/KaizerloveRussia/onnx2torch.git#egg=onnx2torch

Obtaining onnx2torch from git+https://github.com/KaizerloveRussia/onnx2torch.git#egg=onnx2torch
  Updating ./src/onnx2torch clone
  Running command git fetch -q --tags
  Running command git reset --hard -q ad46e9224f77ccaa06e5a327bc7269aac0778aba
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for onnx2torch (pyproject.toml) ... done
  Created wheel for onnx2torch: filename=onnx2torch-1.5.14-0.editable-py3-none-any.whl size=14406 sha256=637594e4b0aeb35cd416370da14a0b8d776976020bd7737f39017a6b74a3fc60
  Stored in directory: /tmp/pip-ephem-wheel-cache-90hkn4mx/wheels/f9/d1/a3/b82562ab7b0402505d5ec31e97cef459ba9bb1251fcc238b54
Successfully built onnx2torch
  Attempting uninstall: onnx2torch
    Found existing installation: onnx2torch 1.5.14
    Uninstalling onnx2torch-1.5.14:
      Successfully uninstalled onnx2

In [3]:
model = models.detection.retinanet_resnet50_fpn(pretrained=True)
model.eval()
input_data = torch.randn(1, 3, 224, 224)
# Запуск модели
output = model(input_data)
print(output)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RetinaNet_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=RetinaNet_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[{'boxes': tensor([[197.9970,   0.9527, 222.7083,  14.3276],
        [  1.0867,   4.4256,  48.4782,  28.9299],
        [ 56.3968, 151.0832,  99.3880, 171.6880],
        [  1.8909,  33.5520,  41.6863,  55.4489],
        [  5.5529, 140.2593,  43.6604, 160.3115],
        [  5.5529, 140.2593,  43.6604, 160.3115],
        [182.9208, 112.3886, 217.9838, 137.4854],
        [ 55.7164, 151.0701,  96.1386, 171.0518],
        [  1.8909,  33.5520,  41.6863,  55.4489],
        [190.4752, 173.2403, 222.0152, 201.1984],
        [  1.8909,  33.5520,  41.6863,  55.4489],
        [120.2304,   4.9470, 167.0288,  32.4612],
        [182.9208, 112.3886, 217.9838, 137.4854],
        [151.7597,  53.6294, 193.0419,  74.5306],
        [154.4435,  60.0895, 198.3574,  80.2292],
        [ 65.9652,  89.8515, 101.1847, 108.9081],
        [ 80.2096, 177.7173, 130.2388, 198.8594],
        [197.9970,   0.9527, 222.7083,  14.3276],
        [  9.4728, 134.1590,  50.7110, 156.1902],
        [  5.5529, 140.2593,  43.6604, 

In [4]:
torch.onnx.export(model, input_data, "retinanet_resnet50_fpn.onnx", opset_version=13)

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4009: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  (torch.floor((input.size(i + 2).float() * torch.tensor(scale_factors[i], dtype=torch.float32)).float()))
/usr/local/lib/python3.10/dist-packages/torchvision/ops/boxes.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes_x = torch.min(boxes_x, torch.tensor(width, dtype=boxes.dtype, device=boxes.device))
/usr/local/lib/python3.10/dist-packages/torchvision/ops/boxes.py:168: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bo

In [5]:
# Загрузка ONNX-модели
onnx_model = onnx.load("retinanet_resnet50_fpn.onnx")
# Создание исполнителя ONNX-модели
ort_session = onnxruntime.InferenceSession("retinanet_resnet50_fpn.onnx")
# Запуск модели из файла ONNX
ort_inputs = {ort_session.get_inputs()[0].name: input_data.numpy()}
ort_outs = ort_session.run(None, ort_inputs)
print("Результат запуска модели RetinaNet в ONNX:", ort_outs)

Результат запуска модели RetinaNet в ONNX: [array([[197.99696  ,   0.9527337, 222.70833  ,  14.327539 ],
       [  1.086694 ,   4.4255633,  48.47818  ,  28.92985  ],
       [ 56.3969   , 151.08322  ,  99.38811  , 171.68797  ],
       [  1.890925 ,  33.55204  ,  41.686325 ,  55.448906 ],
       [  5.552894 , 140.25935  ,  43.660355 , 160.31148  ],
       [  5.552894 , 140.25935  ,  43.660355 , 160.31148  ],
       [182.92075  , 112.3886   , 217.98378  , 137.48543  ],
       [ 55.716423 , 151.0701   ,  96.13868  , 171.05176  ],
       [  1.890925 ,  33.55204  ,  41.686325 ,  55.448906 ],
       [190.47523  , 173.24026  , 222.01523  , 201.19832  ],
       [  1.890925 ,  33.55204  ,  41.686325 ,  55.448906 ],
       [120.23033  ,   4.946999 , 167.02882  ,  32.461143 ],
       [182.92075  , 112.3886   , 217.98378  , 137.48543  ],
       [151.75967  ,  53.6294   , 193.04193  ,  74.530594 ],
       [154.44345  ,  60.089546 , 198.35751  ,  80.22918  ],
       [ 65.96517  ,  89.85156  , 101.184

In [6]:
#Конвертация в PyTorch
from onnx2torch.converter import convert
torch_model = convert("retinanet_resnet50_fpn.onnx")

NotImplementedError: Converter is not implemented (OperationDescription(domain='', operation_type='If', version=13))